In [4]:
import numpy as np


from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [5]:
def load_data():
    print('Loading data...')
    loaded = np.load('data/A00-139_first_10000.npz')
    boards = loaded['arr_0']
    targets = loaded['arr_1']
    print('Data loaded.  Splitting data...')
    b_train, b_test, t_train, t_test = train_test_split(boards, targets, test_size=0.25)
    print('Returning output...')
    return b_train, b_test, t_train, t_test

In [6]:
b_train, b_test, t_train, t_test = load_data()

Loading data...
Data loaded.  Splitting data...
Returning output...


### Applying Convolutional Neural Networks to the problem

Modeled off of the approach by Oshri and Khandwala, we would like 

In [ ]:
def 